In [26]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time
import traceback

In [2]:
state = 'va'

In [5]:
with open('../data/state_county.json', 'r') as f:
    scj = json.load(f)
len(scj)

1585

In [8]:
state_files = [v for v in scj if v.split('_')[0] == state]
len(state_files)

133

In [30]:
def match_fips(block_df):
    matched_fips = []
    pbar = tqdm(block_df['coor'])
    error_count = 0
    for coor in pbar:
        element = None
        try:
            req = "https://geo.fcc.gov/api/census/area?lat={lat}&lon={lon}%09&censusYear=2020&format=json".format(lat=coor[1], lon=coor[0])
            r = requests.get(req)
            j = r.json()
            element = j['results'][0]['block_fips']
        except Exception as e:
            print(traceback.format_exc())
            error_count += 1
            time.sleep(3)
            pbar.set_description('error: %s' % error_count)
        matched_fips.append(element)
    block_df['matched_fips'] = matched_fips
    return block_df

In [31]:
for sc in state_files:
    scdf = pd.read_csv('../data/%s.csv.xz'%sc)
    scdf['coor'] = scdf.apply(lambda x: (float(x['longitude']), float(x['latitude'])), axis=1)
    fdf = match_fips(scdf)
    fdf.to_csv('../data/with_fips/%s.csv.xz'%sc, index=False)

(-75.45842745992478, 37.952992618885695):   0%|                 | 41/26422 [00:07<1:21:03,  5.42it/s]


KeyboardInterrupt: 

In [21]:
scdf

,state,county,zip,longitude,latitude,address,id,coor
0,va,accomack,23420,-75.782732,37.586385,"33282 grant street,painter,va,23420",va_accomack,"(-75.78273194373745, 37.586384624451455)"
1,va,accomack,23480,-75.687351,37.610529,"7 richardson avenue,wachapreague,va,23480",va_accomack,"(-75.68735058936421, 37.610528599630726)"
2,va,accomack,23480,-75.689122,37.610878,"17 richardson avenue,wachapreague,va,23480",va_accomack,"(-75.68912243182051, 37.6108780537332)"
3,va,accomack,23480,-75.687581,37.610198,"40 brooklyn avenue,wachapreague,va,23480",va_accomack,"(-75.68758139979836, 37.61019849733271)"
4,va,accomack,23480,-75.686306,37.610433,"1 richardson avenue,wachapreague,va,23480",va_accomack,"(-75.6863059726563, 37.61043334427158)"
...,...,...,...,...,...,...,...,...
26417,va,accomack,23417,-75.810774,37.709995,"15227 broadway road,onancock,va,23417",va_accomack,"(-75.81077434653461, 37.709995318705616)"
26418,va,accomack,23410,-75.761638,37.653465,"28361 beacon road,melfa,va,23410",va_accomack,"(-75.76163822795358, 37.653464754699584)"
26419,va,accomack,23356,-75.419386,37.989802,"3238 dock court,greenbackville,va,23356",va_accomack,"(-75.41938613419619, 37.98980185736946)"
26420,va,accomack,23356,-75.422857,37.989581,"3373 captains corr,greenbackville,va,23356",va_accomack,"(-75.42285705018088, 37.9895806209073)"
